### Libraries

In [3]:
import numpy as np
import sympy as sp

### Lagrange approach for equality constraints optimization problem in one variable `x`

In [2]:
def lagrange_opt(of, c):
    """
    Implements Lagrange multiplier for equality constraint optimization problem in one variable `x`.

    Parameters:
        of (string): Objective function.
        c (list): Constraints.

    Returns:
        optimal (int): Optimal solution (x).
    """
    # Define the objective function
    f = sp.sympify(of)

    # Define the list of constraints
    constraints = []
    for constraint in c:
        eq = constraint.split('=')
        constraints.append(sp.sympify(eq[0]) - sp.sympify(eq[1]))

    # Define Lagrange multipliers
    lagrange_multipliers = [sp.symbols(f'lambda{i}') for i in range(len(constraints))]

    # Define the Lagrangian function
    lagrangian = f + sum([lam * constraint for lam, constraint in zip(lagrange_multipliers, constraints)])
    
    variables = [sp.symbols('x')] + lagrange_multipliers

    # Compute gradients w.r.t. the variables
    gradients = [sp.diff(lagrangian, var) for var in variables]

    # Solve the system of equations
    solution = sp.solve(gradients, variables)
    
    # Extract x values from solutions
    points = [point[0] for point in solution]
    
    # Substitute with the points in the objective function
    subs = [f.subs({sp.symbols('x'): point}) for point in points]
    
    # Get the point which minimizes the function
    i = subs.index(min(subs))
    optimal = points[i]

    return optimal

#### Example  

In [3]:
objective_function = 'x**2 + 3*x + 5'
constraints = ['x**2 - 4 = 0']
solution = lagrange_opt(objective_function, constraints)
print("Optimal solution:", solution)

Optimal solution: -2


### Lagrange approach for equality constraints optimization problem in two variables `x` and `y`

In [4]:
def lagrange_opt_xy(of, c, maximize=False):
    """
    Implements Lagrange multiplier for equality constraint optimization problem in two variable `x` and `y`.

    Parameters:
        of (string): Objective function.
        c (list): Constraints.
        maximize (boolean): default value is False.

    Returns:
        optimal (tuple): Optimal solution (x,y).
    """
    # Define the objective function
    f = sp.sympify(of)

    # Define the list of constraints
    constraints = []
    for constraint in c:
        eq = constraint.split('=')
        constraints.append(sp.sympify(eq[0]) - sp.sympify(eq[1]))

    # Define Lagrange multipliers
    lagrange_multipliers = [sp.symbols(f'lambda{i}') for i in range(len(constraints))]

    # Define the Lagrangian function
    lagrangian = f + sum([lam * constraint for lam, constraint in zip(lagrange_multipliers, constraints)])
    
    variables = [sp.symbols('x'),sp.symbols('y')] + lagrange_multipliers

    # Compute gradients w.r.t. the variables
    gradients = [sp.diff(lagrangian, var) for var in variables]

    # Solve the system of equations
    solution = sp.solve(gradients, variables)
    
    # Extract x,y values from solutions
    points = [(point[0],point[1]) for point in solution] 
    
    # Substitute with the points in the objective function
    subs = [f.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]}) for point in points]
    
    # Get the point which minimizes/maximizes the function
    if maximize:
        optimal = points[np.argmax(subs)]
    else:
        optimal = points[np.argmin(subs)]    

    return optimal

In [5]:
of = "2*x**2 +5"
f = sp.sympify(of)
f

2*x**2 + 5

#### Example 1 (slide 12 from lecture)

In [6]:
objective_function = 'x + 2*y'
constraints = ['x**2 + y**2 = 4']
solution = lagrange_opt_xy(objective_function, constraints)
print("Optimal solution:", solution)

Optimal solution: (-2*sqrt(5)/5, -4*sqrt(5)/5)


#### Example 2 (slide 14 from lecture)

In [7]:
objective_function = 'x^2 * y'
constraints = ['x**2 + y**2 = 1']
solution = lagrange_opt_xy(objective_function, constraints, maximize=True)
print("Optimal solution:", solution)

Optimal solution: (-sqrt(6)/3, sqrt(3)/3)


### Lagrange approach for inequality constraints optimization problem in two variables `x` and `y`

In [8]:
def lagrange_opt_xy_ineq(of, c, maximize=False):
    """
    Implements Lagrange multiplier for inequality constraint optimization problem.

    Parameters:
        of (string): Objective function.
        c (list): Constraints in the form `h(x,y) <= 0 `.
        maximize (boolean): default value is False.

    Returns:
        optimal (tuple): Optimal solution (x,y).
    """
    # Define the objective function
    f = sp.sympify(of)
    
    # Define the list of constraints
    constraints = [sp.sympify(constraint) for constraint in c]

    # Define Lagrange multipliers
    lagrange_multipliers = [sp.symbols(f'lambda{i}') for i in range(len(constraints))]

    # Define the Lagrangian function
    lagrangian = f + sum([lam * constraint for lam, constraint in zip(lagrange_multipliers, constraints)])
    
    variables = [sp.symbols('x'),sp.symbols('y')]

    # Compute gradients w.r.t. the variables
    gradients = [sp.diff(lagrangian, var) for var in variables]
    
    eqs = gradients + [lam * constraint for lam, constraint in zip(lagrange_multipliers, constraints)]
    varss = variables + lagrange_multipliers

    # Solve the system of equations
    solution = sp.solve(eqs, varss)
        
    # Remove the points that doesn't satisfy the lambda condition
    points = []
    if maximize:
        for sol in solution:
            if all(np.asarray(sol[2:]) <= 0):
                points.append((sol[0],sol[1]))      
    else: 
        for sol in solution:
            if all(np.asarray(sol[2:]) >= 0):
                points.append((sol[0],sol[1]))
                
    # Remove the points that doesn't satisfy the constraints
    final_points = []
    for point in points:
        satisfies_constraints = True
        for con in constraints:
            # Substitute x and y in the constraint
            substituted_con = con.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]})
            # Check if the constraint is <= 0
            if substituted_con > 0:
                satisfies_constraints = False
                break  # Break the loop if any constraint is not satisfied
        # If the point satisfies all constraints, add it to final_points
        if satisfies_constraints:
            final_points.append(point)

    
    # Substitute with the points in the objective function
    subs = [f.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]}) for point in final_points]
    
    # Get the point which minimizes the function
    if maximize:
        optimal = final_points[np.argmax(subs)]
    else:
        optimal = final_points[np.argmin(subs)]  

    return optimal

#### Example 1 (question 3 in the sheet)

In [9]:
objective_function = 'x * y'
constraints = ['2 - x - y', 'x - y']
solution = lagrange_opt_xy_ineq(objective_function, constraints)
print("Optimal solution:", solution)

Optimal solution: (1, 1)


#### Example 2 (question 4 in the sheet)

In [10]:
objective_function = 'x^2 + y^2 + x*y - 3*x'
constraints = ['-x', '-y']
solution = lagrange_opt_xy_ineq(objective_function, constraints)
print("Optimal solution:", solution)

Optimal solution: (3/2, 0)


### Assignment

#### Implement the lagrange approach to solve optimization problems with mixed constraints (equality and inequality)
- Edit the code above to fit your goal
- Don't use other libararies, write the code from scratch 
- Use your code to solve questions 1 and 2 from the sheet
- Deliverables: .py or .ipynb file + screenshots of the code and the output